In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.insert(0, '../inc')


In [2]:
# Load raw lodging list
#df = pd.read_csv('../DABA/서울시 관광숙박업 인허가 정보.csv', encoding='cp949')
df = pd.read_csv('../DABA/서울시 숙박업 인허가 정보.csv', encoding='cp949')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6664 entries, 0 to 6663
Data columns (total 47 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   개방자치단체코드   6664 non-null   int64  
 1   관리번호       6664 non-null   object 
 2   인허가일자      6664 non-null   object 
 3   인허가취소일자    0 non-null      float64
 4   영업상태코드     6664 non-null   int64  
 5   영업상태명      6664 non-null   object 
 6   상세영업상태코드   6664 non-null   int64  
 7   상세영업상태명    6664 non-null   object 
 8   폐업일자       4173 non-null   object 
 9   휴업시작일자     0 non-null      float64
 10  휴업종료일자     0 non-null      float64
 11  재개업일자      0 non-null      float64
 12  전화번호       6262 non-null   object 
 13  소재지면적      6658 non-null   object 
 14  소재지우편번호    6653 non-null   object 
 15  지번주소       6653 non-null   object 
 16  도로명주소      3986 non-null   object 
 17  도로명우편번호    3821 non-null   float64
 18  사업장명       6664 non-null   object 
 19  최종수정일자     6664 non-null   object 
 20  데이터갱신구분 

In [5]:
# select item
sel_colum = [2,3,4,8,9,10,11,15,16,17,18,21,22,23,24]
sel_df = df.iloc[:,sel_colum]
sel_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6664 entries, 0 to 6663
Data columns (total 15 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   인허가일자    6664 non-null   object 
 1   인허가취소일자  0 non-null      float64
 2   영업상태코드   6664 non-null   int64  
 3   폐업일자     4173 non-null   object 
 4   휴업시작일자   0 non-null      float64
 5   휴업종료일자   0 non-null      float64
 6   재개업일자    0 non-null      float64
 7   지번주소     6653 non-null   object 
 8   도로명주소    3986 non-null   object 
 9   도로명우편번호  3821 non-null   float64
 10  사업장명     6664 non-null   object 
 11  데이터갱신일자  6664 non-null   object 
 12  업태구분명    6664 non-null   object 
 13  좌표정보(X)  5931 non-null   float64
 14  좌표정보(Y)  5931 non-null   float64
dtypes: float64(7), int64(1), object(7)
memory usage: 781.1+ KB


In [6]:
# .. 카카오API로 위경도 삽입, pyproj로 시도하였으나 값이 약간 다름

import requests # 온라인으로 보내거나 받아올 때 사용, 파이썬으로 웹사이트 연동할 때
from urllib.parse import quote # url에서 한글 코드를 변환


# API 키 가져오기
filename = 'C:/Users/tjoeun/Desktop/수업자료/수업_실습/API/카카오API.txt'
with open(filename) as f:
    api_key = f.read().strip()
    
len(api_key)  # 32글자가 되야됨, 파일키 정상 문자열개수
     
search_url = 'https://dapi.kakao.com/v2/local/search/address.json'
lat_list, lon_list, err_list = [], [], []
for i in sel_df.index:
#for i in [37]:
    print(sel_df.지번주소[i])
    temp = quote(sel_df.지번주소[i])
    
    url = f'{search_url}?query={temp}'
    result = requests.get(url, headers={"Authorization": f'KakaoAK {api_key}'}).json()
    # print(result)
    # print(i, float(result['documents'][0]['x']), float(result['documents'][0]['y']))
    try:
        lon_list.append(float(result['documents'][0]['x']))
        lat_list.append(float(result['documents'][0]['y']))
    except:
        lon_list.append(np.NAN)
        lat_list.append(np.NAN)
        err_list.append((i,sel_df.지번주소[i]))
    
sel_df['위도'] = lat_list
sel_df['경도'] = lon_list
sel_df.head()

TypeError: quote_from_bytes() expected bytes

In [7]:
print(err_list)
print(len(err_list))


[(37, '서울특별시 강남구 논현동 6 6-1'), (44, '서울특별시 강서구 방화동 886 외 6필지'), (73, '서울특별시 관악구 남현동 1061-2 3~7층'), (89, '서울특별시 중구 순화동 214-0 외 1필지'), (94, '서울특별시 성동구 도선동 58-0번지 '), (95, '서울특별시 강북구 수유동 174-32 외1필지(174-19)'), (117, '서울특별시 마포구 합정동 381-17번지 '), (150, '서울특별시 종로구 수송동 51-8 '), (175, '서울특별시 종로구 당주동 29 '), (194, '서울특별시 중구 을지로6가 17-2번지 케레스타 빌딩 18층~20층 '), (212, '서울특별시 중구 북창동 93-99 '), (227, '서울특별시 중구 회현동1가 194-19 '), (233, '서울특별시 중구 광희동2가 360번지 3-7층 '), (238, '서울특별시 중구 광희동2가 43-1 3~6층 '), (266, '서울특별시 중구 태평로1가 51-1 '), (274, '서울특별시 중구 순화동 151번지 외 1필지 C동 '), (280, '서울특별시 용산구 이태원동 119-24번지 119-24,25,108-9,123-10~15,116-1,116-5,112-7,112-15 '), (281, '서울특별시 용산구 이태원동 34-69 36-69, 34-159, 44-64'), (294, '서울특별시 성동구 도선동 72번지 외 2필지(73, 74번지) '), (299, '서울특별시 광진구 광장동 산 21-0번지 '), (315, '서울특별시 광진구 광장동 산 21-0'), (332, '서울특별시 성북구 동선동1가 92-20 외 3필지(92-21, 92-26, 92-27)'), (356, '서울특별시 서대문구 창천동 31-92 8,9,10층'), (401, '서울특별시 강서구 화곡동 918-23번지 외 2필지 '), (447, '서울특별시 영등포구 당산동3가 136번지 외 1필지 '), (574, '서울특별시 강남구 논현동

In [11]:
# 사업장명 컬럼 맨 앞으로 
sel_df2 = sel_df[['사업장명','지번주소','도로명주소','위도','경도','인허가일자','인허가취소일자','영업상태코드','폐업일자','휴업시작일자','휴업종료일자','재개업일자']]
sel_df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 610 entries, 0 to 609
Data columns (total 12 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   사업장명     610 non-null    object 
 1   지번주소     610 non-null    object 
 2   도로명주소    602 non-null    object 
 3   위도       583 non-null    float64
 4   경도       583 non-null    float64
 5   인허가일자    610 non-null    object 
 6   인허가취소일자  1 non-null      float64
 7   영업상태코드   610 non-null    int64  
 8   폐업일자     136 non-null    object 
 9   휴업시작일자   9 non-null      object 
 10  휴업종료일자   9 non-null      object 
 11  재개업일자    0 non-null      float64
dtypes: float64(4), int64(1), object(7)
memory usage: 57.3+ KB


In [118]:
sel_df2.to_csv("../DAIN/crw_list_add.txt")

In [119]:
err_df = pd.DataFrame(err_list, columns=['index','위경도 missing'])
err_df.to_csv("../DAIN/err_list_add.txt")